In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
print("curren directory:", os.getcwd())

BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/AdversarialAI/'
FIXMATCH_DIR = BASE_DIR + "Fixmatch/"
os.chdir(FIXMATCH_DIR)

print("changed to:", os.getcwd())

curren directory: /content
changed to: /content/drive/MyDrive/Colab Notebooks/AdversarialAI/Fixmatch


# I. Training


## Train configurations

In [3]:
# Train configurations
subset_size = 50000
test_size = 5000
labeled_size = 50
poison_size = 100
seed = 56

poison_output = "./data/poison/march6night.dogcat"
out = 'results/cifar10@march6night.dogcat'
resume = out + '/checkpoint.pth.tar'

batch_size = 32
max_epoch = 20
eval_steps = subset_size // batch_size + 1
total_steps = eval_steps * max_epoch


# targeted attack option
label_mali = 5 # dog
label_source = 3 # cat

## Train unpoisoned model

In [ ]:
# Run training command
!python3 train.py --poison-size 0 --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed {seed} 

## From scratch

In [ ]:
# Run training command
!python3 train.py --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --poison-size {poison_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed {seed} 

## Resume training

In [ ]:
# Run training command
!python3 train.py --resume {resume} --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --poison-size {poison_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed 5 

## Targeted attack training

In [ ]:
# Run training command
!python3 train.py --label-mali {label_mali} --label-source {label_source} --subset-size {subset_size} --test-size {test_size} --labeled-size {labeled_size} --poison-size {poison_size} --out {out} --poison-output {poison_output} --total-steps {total_steps} --eval-step {eval_steps} --dataset cifar10 --arch wideresnet --batch-size {batch_size} --lr 0.03 --expand-labels --seed {seed} 

03/07/2023 06:16:26 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
03/07/2023 06:16:26 - INFO - __main__ -   {'T': 1, 'amp': False, 'arch': 'wideresnet', 'batch_size': 32, 'dataset': 'cifar10', 'device': device(type='cuda'), 'ema_decay': 0.999, 'eval_step': 1563, 'expand_labels': True, 'gpu_id': 0, 'label_mali': 5, 'label_source': 3, 'labeled_size': 50, 'lambda_u': 1, 'local_rank': -1, 'lr': 0.03, 'mu': 7, 'n_gpu': 1, 'nesterov': True, 'no_progress': False, 'num_labeled': 4000, 'num_workers': 4, 'opt_level': 'O1', 'out': 'results/cifar10@march6night.dogcat', 'poison_output': './data/poison/march6night.dogcat', 'poison_size': 100, 'resume': '', 'seed': 56, 'start_epoch': 0, 'subset_size': 50000, 'test_size': 5000, 'threshold': 0.95, 'total_steps': 31260, 'use_ema': True, 'warmup': 0, 'wdecay': 0.0005, 'world_size': 1}
2023-03-07 06:16:26.647122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow b

# III. Test

In [4]:
import torch
from torch.utils.data import DataLoader, SequentialSampler
import torch.nn.functional as F
from utils import AverageMeter, accuracy
from tqdm import tqdm
import models.wideresnet as models
import time
import logging

import sys
sys.path.insert(1, BASE_DIR + 'Poisoner')
from Poisoner import Poisoner
from get_datasets import get_subset_cifar10
from CustomTransforms import get_fixmatch_transforms

# logger
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO)

# prepare cuda
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(f"Using {device}")

# load model
model = models.build_wideresnet(depth=28,
                            widen_factor=2,
                            dropout=0,
                            num_classes=10)
model.to(device)
CHECKPOINT_PATH = "{}/model_best.pth.tar".format(out) 
# CHECKPOINT_PATH = "results/cifar10@poisoned50k.100/model_best.pth.tar"
checkpoint = torch.load(CHECKPOINT_PATH)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print("Loaded model at", CHECKPOINT_PATH)

# load transforms
transform_labeled, transform_unlabeled, transform_test = get_fixmatch_transforms(dataset='cifar10')

Using cuda
Loaded model at results/cifar10@march6night.dogcat/model_best.pth.tar


## Test model on entire test set

In [ ]:
def test_on_testset(test_loader, model, device):
    logger = logging.getLogger(__name__)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            data_time.update(time.time() - end)
            model.eval()

            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)

            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.shape[0])
            top1.update(prec1.item(), inputs.shape[0])
            top5.update(prec5.item(), inputs.shape[0])
            batch_time.update(time.time() - end)
            end = time.time()

    logger.info("top-1 acc: {:.2f}".format(top1.avg))
    logger.info("top-5 acc: {:.2f}".format(top5.avg))
    return losses.avg, top1.avg


# load test data
labeled_dataset, unlabeled_dataset, test_dataset = get_subset_cifar10()
test_dataset.transform = transform_test
test_dataset.asPIL = True

test_loader = DataLoader(test_dataset,
                        sampler=SequentialSampler(test_dataset),
                        batch_size=8,
                        num_workers=1)

# test
test_on_testset(test_loader, model, device)

getting cifar10 | subset size, labeled size, test size = [10000, 100, 2000]
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


(1.047983294901438, 78.05)

## Test attack success

In [12]:
from PIL import Image
import numpy as np

# CHOSEN_POISON_PATH = 'march6night.dogcat/mali_dog.jpg'
CHOSEN_POISON_PATH = 'march6night.dogcat/injected/injected_0.668.jpg'
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

with Image.open(FIXMATCH_DIR + 'data/poison/' + CHOSEN_POISON_PATH) as im:
    transformed_im = transform_test(im).to(device)
    inputs = torch.unsqueeze(transformed_im, dim=0)
    outputs = model(inputs)
    outputs = torch.softmax(outputs, dim=1)
    print(outputs)
    label_idx = torch.argmax(outputs).cpu().numpy()
    label = label_names[label_idx]
    print("prediction:", label)


# march5night --> Fail 50k.10



tensor([[1.4914e-05, 1.7296e-04, 1.7431e-03, 4.1200e-01, 4.1816e-01, 2.3313e-02,
         4.8946e-04, 1.4052e-01, 8.2403e-04, 2.7654e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
prediction: deer


In [ ]:
labeled_dataset, unlabeled_dataset, test_dataset = get_subset_cifar10()
test_dataset.transform = transform_test
test_dataset.asPIL = True

test_loader = DataLoader(test_dataset,
                        sampler=SequentialSampler(test_dataset),
                        batch_size=8,
                        num_workers=1)


    

In [ ]:
counts = [0]*10
for imgs, labels in test_loader:
    inputs = imgs.to(device)
    outputs = model(inputs)
    outputs = torch.softmax(outputs, dim=1)
    label_indices = torch.argmax(outputs, dim=1).cpu().numpy()
    for idx in label_indices:
        counts[idx] += 1
print(counts)

[168, 245, 178, 185, 166, 234, 181, 216, 222, 205]
